In [83]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import Bio
import sys
import os
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import *
from Bio.Alphabet import IUPAC

In [93]:
"""DataFrame Initialization
One for the original sequence reference
and the other for oligos

Oligo generation and original sequence reference recording
Iterate through the number of sequences 
to update the original sequence reference dataframe with
the sequence ID, the original sequence, 
its reverse and complementary sequence, and its sequence length 

For each sequence, iterate per 60 bp 
to update the oligo dataframe with
the target name that is the sequence ID, 
the probe name that is the sequence ID 
plus the oligo number for that sequence ID,
and the 60 bp or shorter oligo sequence itself 
in the 5' to 3' direction

Note that oligos less than 4 bp are elimiated

Export the dataframes to an Excel file
One dataframe per sheet with adjustments such as
the column widths, alignments, and text fonts.
"""

def SDRNA_Oligos(inputfolder, fastafile, strainname, details):
    pathname = Path(inputfolder + "/")
    fastafile = fastafile + ".fasta"
    inputfile = pathname / fastafile
    
    d = {"SeqID": "", "ConSeq": "", "RevComp": "", "SeqLen": []}
    df = pd.DataFrame(data=d)

    opsheet = {"Target": [],
               "Probe Name": [],
               "Sequence (5'-3')": []}
    opf = pd.DataFrame(data=opsheet)
    opname = ("SDRNA oligos in " + str(strainname) 
             + " " + str(details))
    
    opfasta = opname + ".fasta"
    opfastapath = pathname / opfasta
    f = open(opfastapath, "w")

    col1width = 0
    col2width = 0 # initialize the column width for the probe name
        
    for seq_record in SeqIO.parse(inputfile, "fasta"):
        revcomp = str(seq_record.seq.reverse_complement()[::])
        seqlen = len(seq_record)
        
        targetname = seq_record.id.split("-")[0]
        # targetname = seq_record.id.split("|")[4]
                           
        df = df.append({"SeqID": targetname,
                        "ConSeq": str(seq_record.seq[::]),
                        "RevComp": revcomp,
                        "SeqLen": seqlen}, ignore_index=True)
            
        oligtot = math.ceil(seqlen/60)
        
        for olignumb in range(oligtot):
                                  
            seqstart = olignumb*60
            
            if olignumb != oligtot-1:
                seqend = seqstart+60
            elif olignumb == oligtot-1 and seqlen-seqstart < 7:
                print("Skip the last oligo for " + str(targetname) + 
                      ", as it is shorter than or equal to 6 bp")
                continue
            else:
                seqstart = seqlen-60
                seqend = seqlen
            
            probename = str(targetname) + "-SDRNA-" + str(olignumb+1)
            probeseq = revcomp[seqstart:seqend]
            
            opf = opf.append({"Target": targetname,
                              "Probe Name": probename,
                              "Sequence (5'-3')": probeseq}, ignore_index=True)

            f.write(">%s\n%s\n" % (probename,probeseq))
         
        if len(targetname) > col1width:
            col1width = len(targetname)
        
        if len(probename) > col2width:
            col2width = len(probename)
            
    f.close()
    
    fastaoligname = "SDRNA oligos in " + str(strainname) + " " + str(details)
    
    df["SeqLen"] = df["SeqLen"].astype("int")
    
    opExcel = opname + ".xlsx"
    opExcelpath = pathname / opExcel
    writer = pd.ExcelWriter(opExcelpath, engine='xlsxwriter')
    opf.to_excel(writer, sheet_name="Oligos", 
                 header=True, index=False, index_label=None)
    df.to_excel(writer, sheet_name="Sequences", 
                header=True, index=False, index_label=None)
    workbook  = writer.book
    leftformat = workbook.add_format({"text_wrap": True,
                                      "valign": "vcenter",
                                      "font_name": "Courier New"})
    centerformat = workbook.add_format({"text_wrap": True,
                                        "align": "center",
                                        "valign": "vcenter",
                                        "font_name": "Courier New"})
    
    worksheet1 = writer.sheets["Oligos"]
    worksheet1.set_column("A:A", col1width*2, leftformat)
    worksheet1.set_column("B:B", col2width*2, leftformat)
    worksheet1.set_column("C:C", 80, leftformat)
    
    worksheet2 = writer.sheets["Sequences"]
    worksheet2.set_column("A:A", col1width*2, centerformat)
    worksheet2.set_column("D:D", None, centerformat)
    worksheet2.set_column("B:C", 60, leftformat)

    writer.save()
    writer.close()
    
    return fastaoligname
    
fastaoligname = SDRNA_Oligos("NCBI or Manual Compiling Testing/SC Testing", "SC Compiled Seq", "S. cerevisiae ", "")

Skip the last oligo for RDN5, as it is shorter tha or equal to 6 bp


In [43]:
pathname = Path("SC 7000071044" + "/")
fastafile = "13003958421.gb"
inputfile = pathname / fastafile

for seq_record in SeqIO.parse(inputfile, "gb"):
    chrom12 = str(seq_record.seq[::])
    chrom12rdn37revcomp = seq_record.seq[431334-1:438191]
    rdn37 = chrom12rdn37revcomp.complement()
    

print(rdn37)

TATCATTTATCATTGAATGTATGTAATCATTTACCATGTGAGAATGTGTGATAGTAGGAGTAGCATATAATATTATCTATATATGTTATGTACAAAAATGGGCCTAGTATCTTAAGAATTCTGTTTATTTTAAATATCTCTGAACAAGTCAGATGAAGAGAGATTTGATCCGGGGCCGAGGACGGTCATGGGTGAATCTTTCTTTATTTTTTGTTTAGTCTGTTGTTTCCGAATTAGAGTCGTCTAGCATTGTTGTTCCGATGAGATGACGAATGTTATGGGGCAACATGTAGATTCAGCATATGTTTACTAAATAGGGGTGCGTTTTACTGTAACGTTAAGCGGTCGTTCGTGGGTTCCGGAAAGGCGGTTCACGTGGCAACGATCGGACGATACCAAGTCGCTGCGGTGTTCCTGCGGAATAAGCATAGGTAGATATAACACACCTCGTTTCTTTAGTGGCGCAAGATCGTACCTAAGACTGAATCTCCGCAAGTCGGTATTAGGTCGCCTACCATCGAAGCGCCGTTACGGACTAGTCTGTCGGCGTTTTTGGTTAATAGGCTTACTTGACAAGGAGAGCATGATTCAAGTTAATGATAACGCCATTGTAAGTAGTCATCCCATTTTGATTGGACAGAGTGCTGCCAGATTTGGGTCGAGTGCAAGGGATAATCACCCACTTGTTAGGTTGCGAATGGCTTAAGACGAAGCCATACTATCCTTCTCGGCTGTAGCTTCTTAGTTTTTCGTTACAGCGATACTTGCGAACTGACGGTGTTCGGTCAATAGGGACACCATTGAAAAGACCGTGGAGATCGGAGTTTAAGGCTCCCTGATTTCCTAGCTATCCGGTGTGAAAGTACCAAACATAAGTGTGACTTTTAGTTTTAGTTCCCCCGAAAATGGGAAAACAAGATGACCTCTAAAGACAAGAGGTACTCGGGGGGAATCCTGTAGACGCAATAGCAAATTGTCTACACGGCGGGGTCGGTTTGAG

'   \nprint(chrom12.find("ATCCTGCCAG"))\nprint(chrom12revcomp.find("ATCCTGCCAG"))\nrdn18start = chrom12revcomp.find("ATCCTGCCAG") - 10\nprint(chrom12revcomp[rdn18start:rdn18start+1801])\n'

In [7]:
fasta_string = open(fastaoligname + ".fasta").read()
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
blast_result = open("Blast Total" + fastaoligname + ".xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

In [94]:
fasta_string = open(fastaoligname + ".fasta").read()
result_handle = NCBIWWW.qblast(program="blastn", database="refseq_genomic", sequence=fasta_string, entrez_query="txid4932[ORGN]")
records = NCBIXML.parse(result_handle)
blast_record = records.next()

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < 5:
            print ("***** RECORD ****")
            print "sequence:", alignment.title
            print "E-value:", hsp.expect
            
blast_result = open("Blast " + fastaoligname + ".xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("***** RECORD ****")? (<ipython-input-94-0e6c6039064e>, line 9)

In [89]:
blastpath = Path("NCBI or Manual Compiling Testing/SC Testing/")
blastname = "Blast_SC_CEN_PK113-7D.xml"
blastfile = blastpath / blastname
result_handle = open(blastfile,"r")

blast_records = NCBIXML.parse(result_handle)
blast_records = list(blast_records)
E_VALUE_THRESH = 0.04
for blast_record in blast_records:
    print(blast_record.database_sequences)
    for alignment in blast_record.alignments:
        print(alignment)
        for hsp in alignment.hsps:
            print(hsp)
            print(hsp.expect)
            if hsp.expect < E_VALUE_THRESH:
                print("****Alignment****")
                print("sequence:", alignment.title)
                print("length:", alignment.length)
                print("e value:", hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [90]:
help(blast_record)

Help on Blast in module Bio.Blast.Record object:

class Blast(Header, DatabaseReport, Parameters)
 |  Saves the results from a blast search.
 |  
 |  Members:
 |  descriptions        A list of Description objects.
 |  alignments          A list of Alignment objects.
 |  multiple_alignment  A MultipleAlignment object.
 |  + members inherited from base classes
 |  
 |  Method resolution order:
 |      Blast
 |      Header
 |      DatabaseReport
 |      Parameters
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize the class.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Header:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [ ]:
'''
fasta_string = open(fastaoligname + ".fasta").read()
gbaccn = "Z13005731920"
result_handle = NCBIWWW.qblast(program="blastn", 
                               database="refseq_genomic", 
                               sequence=fasta_string, 
                               entrez_query=gbaccn+"[ACCN]")
blast_result = open(gbaccn + "Blast " + fastaoligname + ".xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()

fasta_string = open("Oligo Output.fasta").read()
result_handle = NCBIWWW.qblast(program="blastn", database="refseq_genomic", sequence=fasta_string, entrez_query="txid889517[ORGN]")
blast_result = open("Blast_SC_CEN_PK113-7D.xml", "w")
blast_result.write(result_handle.read())
blast_result.close()
result_handle.close()
'''